In [1]:
import chess
import chess.engine

import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [2]:
from generate_data import *

In [3]:
class ChessDataset(Dataset):
    """Chess Positions Evaluation Dataset"""

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        position = np.array(self.df.iloc[idx, :-1])
        eval_ = np.array(self.df.iloc[idx, -1]/100)
        
        sample = {'position': torch.from_numpy(position), 'eval': torch.from_numpy(eval_)}

        return sample

In [4]:
dataset = ChessDataset(csv_file='data/stockfish_depth0e.csv')

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.fc1 = nn.Linear(513, 64)
        # self.conv1 = nn.Conv1d(64, 8, 3) #input, output channels, kernel_size
        # self.conv2 = nn.Conv2d(8, 8, 3)
        # self.conv3 = nn.Conv2d(8, 8, 3)
        # self.conv4 = nn.Conv2d(8, 8, 3)
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32,8)
        self.fc4 = nn.Linear(8, 1) 

    def forward(self, x):
        x = self.fc1(x)
        # x = self.conv1(x)
        # x = self.conv2(x)
        # x = self.conv3(x)
        # x = self.conv4(x)   
        x = torch.reshape(x, (-1,)) # flattens into 1d tensor
        x = self.fc2(x)
        x = self.fc3(F.relu(x))
        x = self.fc4(torch.sigmoid(x))
        #x = self.fc1(x)
        #x = self.fc2(x)

        return x

In [6]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

net = Net()#.cuda()

In [7]:
t = torch.tensor([[1., -1.], [1., -1.]])
t = t.cuda()
t.get_device()

0

# TRAINING DATA

In [8]:
#torch.cuda.get_device_name(device)

In [9]:
trainloader = DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=0) 
#num_workers = 0 doesn't use GPU. GPU seems to be a lot slower than CPU,.
#data set is prob too big or my laptop nvidia GPU is just trash

optimizer = optim.Adam(net.parameters(), lr = 0.0009) #lr = 0.001
criterion = nn.MSELoss() 

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['position'].reshape(1, 513).float()#.cuda()
        labels = data['eval'].float()#.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 400))
            running_loss = 0.0

print('Finished Training')

[1,   400] loss: 85.990
[1,   800] loss: 85.381
[1,  1200] loss: 84.308
[1,  1600] loss: 75.546
[1,  2000] loss: 80.239
[1,  2400] loss: 80.691
[1,  2800] loss: 73.755
[1,  3200] loss: 76.299
[1,  3600] loss: 66.727
[1,  4000] loss: 70.641
[1,  4400] loss: 67.789
[1,  4800] loss: 63.940
[1,  5200] loss: 68.291
[1,  5600] loss: 63.618
[1,  6000] loss: 63.015
[1,  6400] loss: 64.562
[1,  6800] loss: 60.853
[1,  7200] loss: 63.819
[1,  7600] loss: 62.458
[1,  8000] loss: 50.339
[1,  8400] loss: 55.391
[1,  8800] loss: 59.422
[1,  9200] loss: 51.539
[1,  9600] loss: 50.754
[1, 10000] loss: 43.774
[1, 10400] loss: 50.894
[1, 10800] loss: 50.856
[1, 11200] loss: 47.482
[1, 11600] loss: 49.281
[1, 12000] loss: 47.288
[1, 12400] loss: 50.070
[1, 12800] loss: 52.276
[1, 13200] loss: 42.666
[1, 13600] loss: 32.977
[1, 14000] loss: 52.046
[1, 14400] loss: 45.100
[1, 14800] loss: 45.705
[1, 15200] loss: 42.867
[1, 15600] loss: 43.828
[2,   400] loss: 35.795
[2,   800] loss: 41.602
[2,  1200] loss:

In [10]:
def compare_eval():
    board = generate_rand_board()
    #print(convert_board(board))
    torch_board = torch.from_numpy(convert_board(board).reshape(1, 513)).float()
    net_eval = net(torch_board)
    sf_eval = stockfish_evaluation(board)
    print(f"NN: {net_eval} Stockfish: {sf_eval/100}")
    

In [11]:
for _ in range(5):
    compare_eval()

NN: tensor([-1.2406], grad_fn=<AddBackward0>) Stockfish: -3.33
NN: tensor([2.9934], grad_fn=<AddBackward0>) Stockfish: -1.76
NN: tensor([13.5828], grad_fn=<AddBackward0>) Stockfish: 1.46
NN: tensor([-0.4147], grad_fn=<AddBackward0>) Stockfish: -0.65
NN: tensor([11.6987], grad_fn=<AddBackward0>) Stockfish: 13.32


In [12]:
# Choosing Best Move with Minimax

In [38]:
def nn_prediction(position: chess.Board):
    board_rep = convert_board(position)
    torch_board = torch.from_numpy(board_rep.reshape(1, 513)).float()
    return net(torch_board)


In [39]:

        

def minimax(position: chess.Board, maximizing_player, depth: int = 5): # alpha: int = -1e9, beta: int = 1e9 
    """
    Minimax with alpha-beta pruning to save time
    """
    #just used minimax from connectx kaggle agent

    alpha = -1000 #could be -np.Inf
    beta = 1000
    if depth == 0 or position.is_game_over():
        return nn_prediction(position)

    if maximizing_player:
        value = -np.Inf
        for move in position.legal_moves:
            pos_copy = position.copy()
            if position.remove_piece_at(move.from_square).color == chess.BLACK:
                position = pos_copy.copy()
                continue
            position.push(move)
            value = max(value, minimax(position, False, depth - 1))
            position.pop()
            alpha = max(alpha, value)
            if beta <= alpha:
                break
        return value

    else:
        value = np.Inf
        for move in position.legal_moves:
            pos_copy = position.copy()
            if position.remove_piece_at(move.from_square).color == chess.WHITE:
                position = pos_copy.copy()
                continue
            position.push(move)
            value = max(value, minimax(position, True, depth - 1))
            position.pop()
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

In [48]:
def find_best_move(position: chess.Board, depth: int = 0):
    best_evaluation = -1000
   
    for move in position.legal_moves:
        print(move)
        value = -1000
        # pos_copy = position.copy()
        # if position.remove_piece_at(move.from_square).color == chess.BLACK:
        #     position = pos_copy.copy()
        #     continue
        print(move)
        position.push(move)
        if depth != 0:
            if position.turn == chess.WHITE:
                value = minimax(position, False, depth - 1) 
            else:
                value = minimax(position, True, depth - 1) 
        position.pop()
        if value > best_evaluation:
            best_evaluation = value
            best_move = move
    return best_move


In [46]:
def play_game():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")    
    while not board.is_game_over():
        print(board)
        nn_move = find_best_move(board,1)
        print(nn_move)
        board.push(nn_move)
        moved = False
        while not moved:
            try:
                board.push_san((input('your move')))
                moved = True
            except:
                moved = False


In [49]:
play_game()

r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
g1h3
g1h3


AssertionError: push() expects move to be pseudo-legal, but got g1h3 in rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKB1R